token[end_digit_index:]# Numeric to word

Converts:
- General number and decimals (Distance, weight, etc)
- Time
    - Hour (enhanced from https://sukhbinder.wordpress.com/2013/12/29/time-in-words-with-python/)
    - Date (format is following from https://en.wikipedia.org/wiki/Date_format_by_country)
    - Year
- Phone number (https://stackabuse.com/validating-and-formatting-phone-numbers-in-python)
- Money (the currency symbols)
    - Find the currency symbols in text:
        - https://www.geeksforgeeks.org/how-to-find-index-of-any-currency-symbols-in-a-given-string/
    - The symbols:
        - https://thefactfile.org/countries-currencies-symbols/ (taken)
        - https://wise.com/gb/blog/world-currency-symbols

TO DO:
- Corner cases check (please write your test cases in this notebook)
- Implementation to NL-augmenter

Further improvements:
- Word to numeric
    - https://pypi.org/project/word2number-i18n/
    - https://pypi.org/project/word2number/
- All lang numeric
    - https://pypi.org/project/fastlangid/

In [1]:
import numeric2word

#### Implementation

In [2]:
# TO DO: Check whether all the desired capabilities are met

transformed_sentences = []
for sentence in [    
                "Please buy me 20 apples", #done
                "Please buy me 20000 apples", #done
                "The accuracy is 99.22", #done
                # Date
                "The deadline is in 2020/01/02", #done
                "The deadline is in 2020/01", #unhandled
                "The deadline is in Jan 2020", #done
                "This is 2020!", #done
                # Time
                "Slow down, it\'s still 5:00", #done
                "Slow down, it\'s still 5:10", #done
                "Slow down, it\'s still 5:29", #done
                "Quick!, it\'s already 23:00", #done
                "Quick!, it\'s already 11:30", #done
                "Quick!, it\'s already 9:45", #done
                "Quick!, it\'s already 1:50", #done
                # Date Time
                "Set up a schedule for Monday, June 5th 2021 at 10:00", #done
                "Your christmas dinner is on 25 December 2021 at 8pm", #done
                "You have an appointment on 14/07/2021 at 6pm", #done
                "You have an appointment on 07/14/2021 at 6pm", #done
                "You have an appointment on 07/07/1991 at 6pm", #done
                # Percentage
                "He only use 20% of my power", #done
                "My phone's battery is less than 10%", #done
                # Phone Number
                "My phone number is +1371893178", #done
                "My phone number is +6287822216501", #done
                "My phone number is 6287822", #done
                "My phone number is 628782", #done
                # Currency
                "The price is $.2",
                "The price is 300.2$", #done
                "The price is USD300.2", #done
                "The price is 300.2USD", #done
                "The price is USD300!@#!", #unhandled
                # SemEval 2019 Task 10: Math Question Answering (https://www.aclweb.org/anthology/S19-2153.pdf) 
                # sticky numbers case: math coefficients
                "Suppose 3x + y = 15, where x is a positive integer. What is the difference between the largest possible value of y and the smallest possible value of x, assuming that y is also a positive integer?", #done
                "At a basketball tournament involving 8 teams, each team played 4 games with each of the other teams. How many games were played at this tournament?", #done
                "The lengths of two sides of a triangle are (x − 2) and (x + 2), where x > 2. Which of the following ranges includes all and only the possible values of the third side y? (A) 0 < y < x (B) 0 < y < 2x (C) 4 < y < 2x", #done
                # ChemistryQA (https://openreview.net/pdf?id=oeHTRAehiFF)
                "At a particular temperature a 2.00 L flask at equilibrium contains 2.80 × 10−4 mol N2, 2.50 × 10−5 mol O2, and 2.00 × 10−2 mol N2O. How would you calculate K at this temperature for the following reaction: N2(g) + O2(g) → N2O(g) ?",
                "Volume of the flask is 2.00L. provided Mole of N2 is 2.80 × 10−4 mol. Mole of O2 is 2.50 × 10−5 mol. Mole of N2O is 2.00 × 10−2 mol. Reaction equation is N2(g) + O2(g) → N2O(g).",
                "How many moles of ammonium nitrate are in 335 mL of 0.425 M NH4NO3?", #done unhandled
                "What is the empirical formula of magnesium chloride if 0.96 g of magnesium combines with 2.84 g of chlorine?", #done
                "How would you write a balanced equation for the combustion of octane, C8H18 with oxygen to obtain carbon dioxide and water?", #done unhandled
                # PubmedQA (https://www.aclweb.org/anthology/D19-1259.pdf)
                "221 patients underwent CABG in our hospital from 2004 to 2007. 14 patients with preoperative AF and 4 patients with concomitant valve surgery", #done
                "The overall incidence of postoperative AF was 26%. Postoperative AF was significantly lower in the Statin group compared with the Non-statin group (16% versus 33%, p=0.005).",
                "30-day mortality was 12.4% in those aged<70 years and 22% in those>70 years (p<0.001).",
                "A total of 4 children aged 5-14 years with a sternal fracture were treated in 2 years, 2 children were hospitalized",
                # SMD / KVRET (https://www.aclweb.org/anthology/2020.findings-emnlp.215/)
                "Ok setting your medicine appointment for 7pm", #done
                "It will be between 20-30F in Alhambra on Friday.",
                "Safeway is located at 452 Arcadia Pl.", #done
                "The closest parking garage is Civic Center Garage, located 4 miles away at 270 Altaire Walk.", #done
                "Valero is at 200 Alester Avenue.", #done
                "Setting reminder for yoga activity with your mother on the 15th at 3pm ", #done
                "You will find Pizza Chicago at 915 Arbol Dr. It is 2 miles away without any traffic", #done
            ]:
    transformed_words = []
    for word in sentence.split():
        transformed_word = numeric2word.recognize_transform(word)
        transformed_words.append(transformed_word)
    transformed_sentence = ' '.join(transformed_words)
    transformed_sentences.append({sentence:transformed_sentence})

for data in transformed_sentences:
    for k,v in data.items():
        print(k)
        print(v)
        print()

Please buy me 20 apples
Please buy me twenty  apples

Please buy me 20000 apples
Please buy me twenty thousand  apples

The accuracy is 99.22
The accuracy is ninety-nine point two two 

The deadline is in 2020/01/02
The deadline is in the second of january twenty twenty

The deadline is in 2020/01
The deadline is in 2020/01

The deadline is in Jan 2020
The deadline is in Jan twenty twenty

This is 2020!
This is twenty twenty!

Slow down, it's still 5:00
Slow down, it's still five o'clock am

Slow down, it's still 5:10
Slow down, it's still ten past five am

Slow down, it's still 5:29
Slow down, it's still twenty nine past five am

Quick!, it's already 23:00
Quick!, it's already eleven o'clock pm

Quick!, it's already 11:30
Quick!, it's already half past eleven am

Quick!, it's already 9:45
Quick!, it's already quarter to ten am

Quick!, it's already 1:50
Quick!, it's already ten to two am

Set up a schedule for Monday, June 5th 2021 at 10:00
Set up a schedule for Monday, June fifth twe